In [1]:
import jax
import jax.numpy as jnp

In [25]:

alpha = jnp.array([[1., 0., 0.], [1., 0., 0.01]])

# Enforce LICQ on the rows of alpha
def barrier_LICQ (alpha) :
    # Normalize rows of alpha
    alpha = alpha / jnp.linalg.norm(alpha, axis=1, keepdims=True)
    # offdiagonal inner products of rows of alpha
    aaT = alpha @ alpha.T - jnp.eye(alpha.shape[0])
    # safe set defined by non unit offdiagonal inner products
    return 1. - jnp.max(aaT) - 1e-5

# # ex_alpha = jax.grad(barrier_LICQ)(alpha)
balpha = barrier_LICQ(alpha)
print(balpha)
J = jnp.eye(alpha.shape[0])
u0 = -J.T@alpha
u0flat = u0.reshape(-1)
kap = 0.1

pLfh, Lfh = jax.jvp(barrier_LICQ, (alpha,), (u0,))
unroll = lambda v : jax.jvp(barrier_LICQ, (alpha,), (v.reshape(alpha.shape),))
pLgh, Lgh = jax.vmap(unroll)(jnp.eye(alpha.size))
print(Lfh)
print(pLgh, Lgh)

print(Lfh + Lgh@u0flat + kap*balpha)

ustar = u0flat - (-Lfh + Lgh@u0flat + kap*balpha)*Lgh.T/(Lgh@Lgh.T)
print(u0flat)
print(ustar)

3.9948693e-05
-1.1920929e-07
[3.9948693e-05 3.9948693e-05 3.9948693e-05 3.9948693e-05 3.9948693e-05
 3.9948693e-05] [-0.0000000e+00 -0.0000000e+00 -9.9995006e-03 -9.9897385e-05
 -0.0000000e+00  9.9985013e-03]
3.7880359e-06
[-1.    0.    0.   -1.    0.   -0.01]
[-1.0000000e+00  0.0000000e+00  2.0134285e-04 -9.9999797e-01
  0.0000000e+00 -1.0201323e-02]


In [ ]:
# Solution to QP